# Imports

In [ ]:
from sklearn.linear_model import LogisticRegression
import torch

# Constants

In [ ]:
DATA_DIR = "./experimental_data/"

# Data

Load data

In [ ]:
acts_exp_resid = torch.load(DATA_DIR + "acts_exp_resid.pt")
acts_resid = torch.load(DATA_DIR + "acts_resid.pt")

Split data